# 1: インポート

# 2: データ読込み

# 3: 特徴量数値化

# 4: ハイパーパラメータの調整

# 5: 実装

# 6: 過学習の有無を確認

# 1: インポート

In [1]:
import pandas as pd
from pandas import Series,DataFrame

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

import numpy as np

import keras
from keras import regularizers
from keras.metrics import mae
from keras import regularizers
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# 2: データの読み込み

In [115]:
data = pd.read_csv(rf'Linux5_8.csv',sep=",", header=0, encoding="CP932")
data.drop(data[["Bug ID"]], axis = 1, inplace=True) 
data.head(5)

,Product,Component,Assignee,Status,Resolution,Summary,Changed
0,Red Hat Enterprise Linux 5,yum,James Antill,CLOSED,NOTABUG,yum won't run,2014/1/21 6:20
1,Red Hat Enterprise Linux 6,pki-core,Matthew Harmsen,CLOSED,ERRATA,pki-symkey jar version incorrect,2011/5/19 13:43
2,Red Hat Enterprise Linux 6,kernel,Abhijith Das,CLOSED,ERRATA,GFS2: allow gfs2 to update quota usage through...,2011/5/19 12:42
3,Red Hat Enterprise Linux 6,virt-v2v,Matthew Booth,CLOSED,ERRATA,virt-v2v should re-introduce support for Windo...,2018/11/14 13:01
4,Red Hat Enterprise Linux 6,webkitgtk,Martin Stransky,CLOSED,NOTABUG,webkit do not cooperate with flash-plugin,2011/1/24 10:17


In [116]:
data.isnull().sum()

Product         0
Component       0
Assignee      909
Status          0
Resolution      0
Summary         0
Changed       120
dtype: int64

In [117]:
# "Assignee"と"Changed"が欠損値の箇所は削除

data.dropna(axis = 0, how = 'any', inplace=True)
data.head(5)

,Product,Component,Assignee,Status,Resolution,Summary,Changed
0,Red Hat Enterprise Linux 5,yum,James Antill,CLOSED,NOTABUG,yum won't run,2014/1/21 6:20
1,Red Hat Enterprise Linux 6,pki-core,Matthew Harmsen,CLOSED,ERRATA,pki-symkey jar version incorrect,2011/5/19 13:43
2,Red Hat Enterprise Linux 6,kernel,Abhijith Das,CLOSED,ERRATA,GFS2: allow gfs2 to update quota usage through...,2011/5/19 12:42
3,Red Hat Enterprise Linux 6,virt-v2v,Matthew Booth,CLOSED,ERRATA,virt-v2v should re-introduce support for Windo...,2018/11/14 13:01
4,Red Hat Enterprise Linux 6,webkitgtk,Martin Stransky,CLOSED,NOTABUG,webkit do not cooperate with flash-plugin,2011/1/24 10:17


# 3: 特徴量の数値化

In [118]:
# バグ修正完了している"CLOSED"なら1，それ以外を0

data["Status"].loc[data["Status"] == "CLOSED"] = 1
data["Status"].loc[data["Status"] != 1] = 0



# ["Product", "Component", "Assignee", "Resolution"]を，frequency encoding

freq_change = data[["Product", "Component", "Assignee", "Resolution"]]

for c in freq_change:
    freq = data[c].value_counts()
    
    # カテゴリの出現回数で置換
    data[c] = data[c].map(freq)
    data[c] = data[c] / len(data)

data.head(5)

,Product,Component,Assignee,Status,Resolution,Summary,Changed
0,0.068157,0.010332,0.003743,1,0.118971,yum won't run,2014/1/21 6:20
1,0.361025,0.005746,0.003005,1,0.414317,pki-symkey jar version incorrect,2011/5/19 13:43
2,0.361025,0.022403,0.000264,1,0.414317,GFS2: allow gfs2 to update quota usage through...,2011/5/19 12:42
3,0.361025,0.001002,0.001265,1,0.414317,virt-v2v should re-introduce support for Windo...,2018/11/14 13:01
4,0.361025,0.000053,0.005851,1,0.118971,webkit do not cooperate with flash-plugin,2011/1/24 10:17


In [119]:
# Changedを昇順に並び替える

data['Changed'] = pd.to_datetime(data['Changed'])
sort_data = data.sort_values(['Changed'])
sort_data.reset_index(drop=True, inplace=True)
sort_data.head(5)

,Product,Component,Assignee,Status,Resolution,Summary,Changed
0,0.361025,0.000053,0.005851,1,0.118971,webkit do not cooperate with flash-plugin,2011-01-24 10:17:00
1,0.361025,0.041537,0.011966,1,0.122397,not all UI widgets are RTL compatible,2011-01-27 21:21:00
2,0.361025,0.002108,0.004322,1,0.118971,[abrt] coreutils-8.4-9.el6: __nanosleep_nocanc...,2011-02-04 14:24:00
3,0.361025,0.022403,0.004428,1,0.118971,ABRT reports crash after boot,2011-02-04 18:43:00
4,0.361025,0.026303,0.001160,1,0.099626,hunspell-en is not in rhel-x86_64-server unlik...,2011-02-04 19:03:00


In [120]:
# 同Componentの修正間隔を求める

import datetime

sort_data["Changed"].astype(str)

Changed = sort_data["Changed"] # 変更日 
Component = sort_data["Component"] # Linuxのバージョン頻度 
sort_data["interval"] = 0
Different = sort_data["interval"]

Component[-1] = 0

for i in range(len(sort_data)): 
    Changed[i] = str(Changed[i]) 
    Changed[i] = Changed[i].replace(" ", "-") 
    Changed[i] = Changed[i].split("-")

    count = 0  # kのループを通過したかを判定
    
    
    # 途中経過
    if ((i % 2000) == 0):
        print(i)


    for j in range(i-1, -2, -1):

        if ((j == i-1) & (j >= 0)):
            for k in range(i-1, -1, -1):
                if ((Component[i] == Component[k]) & (int(Changed[i][1]) == int(Changed[k][1])) & 
                    (int(Changed[i][2]) == int(Changed[k][2])) & (i > 0) & (k > 0)):
                    
                    Different[i] = Different[k]
                    sort_data["interval"][i] = Different[i]
                    count = count + 1
                    break


        if ((Component[i] == Component[j]) & (i > 0) & (count == 0)):
            criterion = datetime.date(int(Changed[j][0]), int(Changed[j][1]), int(Changed[j][2]))
            current = datetime.date(int(Changed[i][0]), int(Changed[i][1]), int(Changed[i][2]))
            Different[i] = (current - criterion).days
            sort_data["interval"][i] = Different[i]
            break

        elif (((j == 0) | (i == 0)) & (count == 0)):
            Different[i] = 0
            sort_data["interval"][i] = 0
            break

0
2000
4000
6000
8000
10000
12000
14000
16000
18000


In [131]:
data = sort_data.drop(sort_data[["Changed"]], axis = 1, inplace=True) 

# 可能なら，形態素解析など行う(現在は文字列の長さ)
for i in range(len(data)):
    data["Summary"][i] = len(data["Summary"][i])


data["target"] = sort_data["Product"] * sort_data["Component"]

data.head(5)

,Product,Component,Assignee,Status,Resolution,Summary,interval,target
0,0.361025,0.000053,0.005851,1,0.118971,41,0,0.000019
1,0.361025,0.041537,0.011966,1,0.122397,37,0,0.014996
2,0.361025,0.002108,0.004322,1,0.118971,101,0,0.000761
3,0.361025,0.022403,0.004428,1,0.118971,29,0,0.008088
4,0.361025,0.026303,0.001160,1,0.099626,75,0,0.009496


In [132]:
# ファイル書き出し
data.to_csv(rf'data.csv',index = False)